### How many 'Forgotten items' baskets are the re in the coop_data.json dataset?

#### Definition  of 'Forgotten items basket': a small basket with 'X' number of products (1-2) bought 'Y' number of days after a 'large' purchase of 'Z' amount of products.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime, timedelta

import os
os.environ["OMP_NUM_THREADS"] = "4"

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#### Identifying the baskets that are considered 'forgotten item' purchases

In [ ]:
def identify_forgotten_item_baskets(df, max_days=0, max_items=1, large_basket_threshold=10):
    # Sort the dataframe by customer_id and next_basket_id (date)
    df = df.sort_values(['customer_id', 'next_basket_id'])
    
    # Convert next_basket_id to datetime
    df['next_basket_date'] = pd.to_datetime(df['next_basket_id'].str.split('_').str[:3].str.join('-'))
    
    # Initialize the forgotten_item_basket column
    df['forgotten_item_basket'] = 0
    
    # Group by customer_id
    for customer_id, group in df.groupby('customer_id'):
        large_basket_date = None
        
        for index, row in group.iterrows():
            actual_basket = str(row['actual_basket']).split(',')
            
            # Check if it's a large basket
            if len(actual_basket) >= large_basket_threshold:
                large_basket_date = row['next_basket_date']
            elif large_basket_date is not None:
                # Check if it's a potential forgotten-item basket
                date_diff = (row['next_basket_date'] - large_basket_date).days
                if 0 <= date_diff <= max_days and len(actual_basket) <= max_items:
                    df.at[index, 'forgotten_item_basket'] = 1
                    large_basket_date = None  # Reset large_basket_date
                elif date_diff > max_days:
                    large_basket_date = None  # Reset large_basket_date if more than max_days have passed
    
    # Drop the temporary next_basket_date column
    df = df.drop(columns=['next_basket_date'])
    
    return df

In [ ]:
# (df, max_days=0, max_items=1, large_basket_threshold=10):

#### SPLIT 10 1 for Each

In [ ]:
path = r"E:\Thesis\Legrottaglie Forgotten Items\Model\Experiments\1fE\tbp\split_10\pred_1fE_10_tbp_2.csv"


df = pd.read_csv(path)